In [ ]:
import numpy as np
import pandas as pd
import random
from joblib import Parallel, delayed
import itertools

In [ ]:
# Optimized function to generate an empty simplicial complex
def generate_empty_simplicial_complex(num_nodes):
    """Generate an empty simplicial complex."""
    nodes = np.arange(1, num_nodes + 1)
    simplicial_complex = {}
    for r in range(1, num_nodes + 1):
        simplices = itertools.combinations(nodes, r)
        for simplex in simplices:
            simplicial_complex[simplex] = 0
    return simplicial_complex

# Optimized function to distribute values for R and S sums
def distribute_value(R_sum, S_sum, nb_of_variables):
    """Distribute values efficiently using NumPy."""
    nb_of_variables_R = random.randint(1, nb_of_variables - 1)
    nb_of_variables_S = nb_of_variables - nb_of_variables_R

    # Generate R and S distributions using numpy for efficiency
    points_R = np.sort(np.random.uniform(0, R_sum, nb_of_variables_R - 1))
    points_R = np.concatenate(([0], points_R, [R_sum]))
    distributed_values_R = np.diff(points_R)

    points_S = np.sort(np.random.uniform(0, S_sum, nb_of_variables_S - 1))
    points_S = np.concatenate(([0], points_S, [S_sum]))
    distributed_values_S = np.diff(points_S)

    # Combine, shuffle, and return the values
    distributed_values = np.concatenate([distributed_values_R, -distributed_values_S])
    np.random.shuffle(distributed_values)

    return distributed_values

# Optimized function to assign MI values
def assign_MI_values(simplicial_complex, R_sum, S_sum):
    """Assign MI values to simplicial complex efficiently."""
    count = sum(1 for key in simplicial_complex if len(key) > 2)
    distributed_values = distribute_value(R_sum, S_sum, count)
    distributed_values_iter = iter(distributed_values)

    for key in simplicial_complex:
        key_length = len(key)
        if key_length == 1:
            simplicial_complex[key] = random.uniform(2, 3)
        elif key_length == 2:
            simplicial_complex[key] = random.uniform(0, 3)
        elif key_length > 2:
            simplicial_complex[key] = next(distributed_values_iter)
    
    return simplicial_complex

# Optimized function to compute TSE complexity
def compute_tse_complexity(TC_dict):
    """Compute TSE complexity efficiently."""
    N = max(len(subset) for subset in TC_dict)  # Maximum subset size
    TSE = 0
    for gamma in range(1, N):
        TC_full = (gamma / N) * TC_dict[tuple(range(1, N + 1))]
        E_TC_gamma = np.mean([TC_dict[subset] for subset in TC_dict if len(subset) == gamma])
        TSE += TC_full - E_TC_gamma
    return TSE

In [ ]:
R_sum = 19
S_sum = 1
nb_of_variables = 10

empty_simplicial_complex = generate_empty_simplicial_complex(nb_of_variables)
Ninfomut_artificial = assign_MI_values(empty_simplicial_complex, R_sum, S_sum)
compute_tse_complexity(Ninfomut_artificial)